<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/DwrtXGradientW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

In [14]:
# parameters for the first layer
W_0 = np.array([[1.05954587,-0.05625762],[-0.03749863,1.09518945]])
b = np.array([[-0.050686,-0.06894291]])

# parameters for the second layer

V_0 = np.array([[3.76921058],[-3.72139955]])
c = np.array([[-0.0148436]])

In [15]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()
features = [
    "x_2dvec",

]
label = "y"

# train test split
X_raw,  Y = data[features].values, data[label].values

#convert string to array
X_raw = np.array([eval(s[0]) for s in X_raw])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(X_raw.shape[0])

[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
10


In [16]:
class SimpleNN(nn.Module):
    def __init__(self, custom_W_0, custom_b, custom_V_0, custom_c):
        super(SimpleNN, self).__init__()

        # Ensure that the custom weights are tensors
        custom_W_0 = torch.tensor(custom_W_0, dtype=torch.float64)
        custom_b = torch.tensor(custom_b, dtype=torch.float64)
        custom_V_0 = torch.tensor(custom_V_0, dtype=torch.float64)
        custom_c = torch.tensor(custom_c, dtype=torch.float64)

        # Set the custom weights and biases
        self.W_0 = nn.Parameter(custom_W_0)
        self.b = nn.Parameter(custom_b)
        self.V_0 = nn.Parameter(custom_V_0)
        self.c = nn.Parameter(custom_c)

    def forward(self, x):
        x = F.sigmoid(torch.add(torch.matmul(x, self.W_0), self.b))
        x = F.sigmoid(torch.add(torch.matmul(x, self.V_0), self.c))
        return x

# Example usage
custom_W_0 = [[0.1, 0.2], [0.3, 0.4]]  # Replace with your own initial values
custom_b = [0.1, 0.2]  # Replace with your own initial values
custom_V_0 = [[0.1], [0.2]]  # Replace with your own initial values
custom_c = [0.1]  # Replace with your own initial values

nn_model = SimpleNN(W_0, b, V_0, c)


In [18]:
# Convert data to PyTorch tensors
X_raw_torch = torch.tensor(X_raw, requires_grad=True)
print(X_raw_torch)
Y_torch = torch.tensor(Y)

# Forward pass
output = nn_model(X_raw_torch)

# Compute loss
loss = -torch.mean(Y_torch * torch.log(output) + (1 - Y_torch) * torch.log(1 - output))
print(loss)
# Compute gradients of the loss w.r.t. weights
#loss.backward()

# Combine and compute the norm of all gradients
#all_grads = torch.cat([model.W_0.grad.flatten(), model.V_0.grad.flatten(), model.b.grad.flatten(), model.c.grad.flatten()])
#grad_norm = torch.norm(all_grads)

# Compute the derivative of the grad_norm with respect to X
#second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]

# If you want to perform gradient descent on X_raw
#learning_rate = 0.01
#X_raw_torch.data -= learning_rate * second_order_grad

tensor([[ 2.8000,  0.4000],
        [ 3.1000,  4.3000],
        [ 0.1000, -3.4000],
        [-4.2000, -3.3000],
        [-0.5000,  0.2000],
        [-2.7000, -0.4000],
        [-3.0000, -4.3000],
        [-0.1000,  3.4000],
        [ 4.2000,  3.2000],
        [ 0.4000, -0.1000]], dtype=torch.float64, requires_grad=True)
tensor(0.5777, dtype=torch.float64, grad_fn=<NegBackward0>)
